# Goal:

provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

np.seterr(all='raise')

DEBUG = True

# Load and Process the Data

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../data/X_norm_1024.csv"
    y_file = "../data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [3]:
NUM_SAMPLES = 1024
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)
betas = np.zeros((10, ))

Using subsampled data to compare Python-C++
Reading in ../data/X_norm_1024.csv, ../data/y_1024.csv 
Train X shape is: (1024, 10)
Train y shape is: (1024, 1)


In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas
    return np.expand_dims(sigmoid(preds), -1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]


def vanilla(betas, epochs, lr, train_x, train_y, silence=False):
    import copy

    theta = copy.deepcopy(betas)

    loss_arr = [0 for _ in range(epochs)]
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        theta = theta - lr * np.squeeze(gradient)
        loss = cost(train_x, train_y, theta)
        if not silence:
            if DEBUG:
                # print(f"Grad: {(gradient.squeeze() * lr).tolist()}")
                print(f"Theta: {theta.tolist()}")
            print(f"Iteration: {i} Loss: {loss}")
        loss_arr[i] = loss

    return loss_arr, theta


def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    # for i in tqdm.trange(epochs):
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        phi_prime = theta - lr * np.squeeze(gradient)
        if i == 0:
            theta = phi_prime
        else:
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime
        loss = cost(train_x, train_y, theta)
        if DEBUG:
            print(f"Grad: {(gradient.squeeze() * lr).tolist()}")
            print(f"Theta: {theta.tolist()}")
            print(f"Phi: {phi.tolist()}")
        print(f"Iteration: {i} Loss: {loss}")
        nesterov_loss[i] = loss

    return nesterov_loss, theta, phi


# Verify Inference

In [5]:
losses, weights = vanilla(betas, 100, lr, train_x, train_y, silence=True)

preds = fwd(train_x, weights)
for i, (y_hat, y) in enumerate(zip(preds, train_y)):
    if i > 10:
        break
    print(y_hat, np.round(y_hat) ,y )



[0.57171037] [1.] [1]
[0.92000353] [1.] [1]
[0.88121884] [1.] [1]
[0.87902254] [1.] [1]
[0.71496151] [1.] [1]
[0.3272676] [0.] [1]
[0.87833188] [1.] [1]
[0.6594513] [1.] [1]
[0.44359946] [0.] [1]
[0.52999849] [1.] [1]
[0.91337266] [1.] [1]


# Callers

In [6]:
losses, weights = vanilla(betas, 100, lr, train_x, train_y)


Theta: [-0.015940504807692296, 0.004660081370379261, -0.018115234375, 0.01513671875, -0.01572265625, 1.5782828282818888e-05, -0.0033203125000000003, 0.0005859375000000001, 0.0009765625, 0.0]
Iteration: 0 Loss: 0.6824088332753198
Theta: [-0.03158854974565724, 0.008797658420500704, -0.035809652796559024, 0.02958523222229394, -0.030782344042775557, -0.0003463170115104892, -0.0061982712479218705, 0.0006561815513320185, 0.001715236710374793, 0.0005105740436275885]
Iteration: 1 Loss: 0.672375487027155
Theta: [-0.046949521297982175, 0.012479705051865525, -0.05309863527278311, 0.04338933125949955, -0.04522976552287428, -0.0010284151579521444, -0.008690928450902902, 0.00028167972350914383, 0.0022594974386068705, 0.0014606141285014235]
Iteration: 2 Loss: 0.6629647982176894
Theta: [-0.062029082328317775, 0.01576520256582197, -0.06999683984169391, 0.056589264959550106, -0.059110655480845604, -0.0019797770964390757, -0.01084856631417933, -0.00047519637500721375, 0.0026470025389547496, 0.00278763482

In [7]:
losses, theta, phi = nesterov(betas, 10, lr, mu, train_x, train_y)

Grad: [0.015940504807692296, -0.004660081370379261, 0.018115234375, -0.01513671875, 0.01572265625, -1.5782828282818888e-05, 0.0033203125000000003, -0.0005859375000000001, -0.0009765625, 0.0]
Theta: [-0.015940504807692296, 0.004660081370379261, -0.018115234375, 0.01513671875, -0.01572265625, 1.5782828282818888e-05, -0.0033203125000000003, 0.0005859375000000001, 0.0009765625, 0.0]
Phi: [-0.015940504807692296, 0.004660081370379261, -0.018115234375, 0.01513671875, -0.01572265625, 1.5782828282818888e-05, -0.0033203125000000003, 0.0005859375000000001, 0.0009765625, 0.0]
Iteration: 0 Loss: 0.6824088332753198
Grad: [0.015648044937964945, -0.0041375770501214425, 0.017694418421559024, -0.01444851347229394, 0.015059687792775554, 0.0003620998397933081, 0.00287795874792187, -7.02440513320184e-05, -0.0007386742103747931, -0.0005105740436275885]
Theta: [-0.03315335423945373, 0.009211416125512848, -0.03757909463871493, 0.031030083569523335, -0.03228831282205311, -0.00038252699548982, -0.00648606712271